# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard

#### Load the pre-trained RESNET-50 model

In [3]:
rnet50 = ResNet50(include_top=False,
                  weights='imagenet',
                  input_shape=(224,224,3),
                  pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [4]:
for layer in rnet50.layers:
    layer.trainable = False

#### Add custom classification layers

In [5]:
# Instantiate the sequential model and add the VGG19 model:
model = Sequential()
model.add(rnet50)

# Add the custom layers atop the VGG19 model:
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).  
Instead of downloading and copying the data to Google Drive, we can use API to download any data from Kaggle.

In [ ]:
# [1] sign in/register to Kaggle account
# [2] from account settings, API "create new token" and download kaggle.json
# [3] upload kaggle.json to colab sample_data

In [10]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "D:\My Teaching\JLDA\MDA 504_F23\Python-Class\kaggle\kaggle.json"

In [13]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [14]:
# Define the batch size:
batch_size=32

In [15]:
# Define the train and validation generators:
train_generator = train_datagen.flow_from_directory(
    directory='./kaggle/train/',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./kaggle/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [16]:
tensorboard = TensorBoard('logs/transfer')

In [17]:
model.fit(train_generator,
          steps_per_epoch=15,
          epochs=20,
          validation_data=valid_generator,
          validation_steps=15,
          callbacks=[tensorboard])

15/15 [==============================] - 35s 2s/step - loss: 1.1762 - accuracy: 0.5150 - val_loss: 0.8673 - val_accuracy: 0.5396
Epoch 3/20
15/15 [==============================] - 35s 2s/step - loss: 1.0812 - accuracy: 0.5365 - val_loss: 0.6814 - val_accuracy: 0.5854
Epoch 4/20
15/15 [==============================] - 33s 2s/step - loss: 1.0368 - accuracy: 0.5343 - val_loss: 0.9645 - val_accuracy: 0.5146
Epoch 5/20
15/15 [==============================] - 33s 2s/step - loss: 1.2067 - accuracy: 0.5258 - val_loss: 1.2695 - val_accuracy: 0.5271
Epoch 6/20
15/15 [==============================] - 35s 2s/step - loss: 1.3985 - accuracy: 0.5193 - val_loss: 0.8195 - val_accuracy: 0.5479
Epoch 7/20
15/15 [==============================] - 33s 2s/step - loss: 1.6214 - accuracy: 0.5279 - val_loss: 1.6891 - val_accuracy: 0.5000
Epoch 8/20
15/15 [==============================] - 33s 2s/step - loss: 1.3115 - accuracy: 0.5429 - val_loss: 0.7416 - val_accuracy: 0.5542
Epoch 9/20
15/15 [=============

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/transfer